## 0 - 疑问
+ 一个普通的python程序在Ｕbuntu ４　CPUs上运行，为什么会自动调用４块ＣＰＵ，而不是只在一块ＣＰＵ上运行？如果它直接调用我的4块ＣＰＵ，那我根本没有必要再去学并行计算．

## 1 - 进程线程基础
Reference: https://www.liaoxuefeng.com/wiki/0014316089557264a6b348958f449949df42a6d3a2e542c000/0014319272686365ec7ceaeca33428c914edf8f70cca383000
+ 多任务：听mp3，看电影，玩游戏一共有３个任务
+ 进程Process, 线程Thread
+ 单核cpu也可以执行多任务：任务1执行0.01秒，切换到任务2，任务2执行0.01秒，再切换到任务3，执行0.01秒……这样反复执行下去。表面上看，每个任务都是交替执行的，但是，由于CPU的执行速度实在是太快了，我们感觉就像所有任务都在同时执行一样。
+ 真正的并行执行多任务只能在多核CPU上实现，但是，由于任务数量远远多于CPU的核心数量，所以，操作系统也会自动把很多任务轮流调度到每个核心上执行。
+ 对于操作系统来说，一个任务就是一个进程（Process），比如打开一个浏览器就是启动一个浏览器进程，打开一个记事本就启动了一个记事本进程，打开两个记事本就启动了两个记事本进程，打开一个Word就启动了一个Word进程。
+ 有些进程还不止同时干一件事，比如Word，它可以同时进行打字、拼写检查、打印等事情。在一个进程内部，要同时干多件事，就需要同时运行多个“子任务”，我们把进程内的这些“子任务”称为线程（Thread）。
+ 每个进程至少要干一件事，所以，一个进程至少有一个线程。当然，像Word这种复杂的进程可以有多个线程，多个线程可以同时执行，多线程的执行方式和多进程是一样的，也是由操作系统在多个线程之间快速切换，让每个线程都短暂地交替运行，看起来就像同时执行一样。当然，真正地同时执行多线程需要多核CPU才可能实现。
+ 如果我们要同时执行多个任务怎么办？有两种解决方案：一种是启动多个进程，每个进程虽然只有一个线程，但多个进程可以一块执行多个任务。还有一种方法是启动一个进程，在一个进程内启动多个线程，这样，多个线程也可以一块执行多个任务。当然还有第三种方法，就是启动多个进程，每个进程再启动多个线程，这样同时执行的任务就更多了，当然这种模型更复杂，实际很少采用。总结一下就是，多任务的实现有3种方式：多进程模式；多线程模式；多进程+多线程模式。
+ 同时执行多个任务通常各个任务之间并不是没有关联的，而是需要相互通信和协调，有时，任务1必须暂停等待任务2完成后才能继续执行，有时，任务3和任务4又不能同时执行，所以，多进程和多线程的程序的复杂度要远远高于单进程单线程的程序。
+ 很多时候，没有多任务还真不行。想想在电脑上看电影，就必须由一个线程播放视频，另一个线程播放音频，否则，单线程实现的话就只能先把视频播放完再播放音频，或者先把音频播放完再播放视频，这显然是不行的。
+ Python既支持多进程，又支持多线程。
+ 线程是最小的执行单元，而进程由至少一个线程组成。如何调度进程和线程，完全由操作系统决定，程序自己不能决定什么时候执行，执行多长时间。

## 2 - 多进程multiprocessing
Reference: https://www.liaoxuefeng.com/wiki/0014316089557264a6b348958f449949df42a6d3a2e542c000/001431927781401bb47ccf187b24c3b955157bb12c5882d000
+ Process创建一个进程p, p可以运行在python内编写的函数
+ p的属性: .start(), .run(), .join()
+ join()方法可以等待子进程结束后再继续往下运行，通常用于进程间的同步。对Pool对象调用join()方法会等待所有子进程执行完毕，调用join()之前必须先调用close()，调用close()之后就不能继续添加新的Process了。
+ 多线程和多进程最大的不同在于，多进程中，同一个变量，各自有一份拷贝存在于每个进程中，互不影响，而多线程中，所有变量都由所有线程共享，所以，任何一个变量都可以被任何一个线程修改，因此，线程之间共享数据最大的危险在于多个线程同时改一个变量，把内容给改乱了。

### 2 - 1 - apply(), Pool.apply(), Pool.apply_async(), Pool.map(), Pool.map_async()
Reference: https://stackoverflow.com/questions/8533318/multiprocessing-pool-when-to-use-apply-apply-async-or-map
+ `apply()` Back in the old days of Python, to call a function with arbitrary arguments, you would use apply `apply(f,args,kwargs)`. `apply()` still exists in Python2.7 though not in Python3, and is generally not used anymore. Nowadays, `f(*args,**kwargs)` is preferred.
+ `Pool.apply()` is like Python `apply()`, except that the function call is performed in a separate process. `Pool.apply()` blocks until the function is completed.
+ `Pool.apply_async()` is also like Python's built-in apply, except that the call returns immediately instead of waiting for the result. An AsyncResult object is returned. You call its `get()` method to retrieve the result of the function call. The `get()` method blocks until the function is completed. Thus, `Pool.apply(func, args, kwargs)` is equivalent to `pool.apply_async(func, args, kwargs).get()`.
+ In contrast to `Pool.apply()`, the `Pool.apply_async()` method also has a callback which, if supplied, is called when the function is complete. This can be used instead of calling `get()`.
+ `Pool.map()` applies the same function to many arguments. However, unlike `Pool.apply_async()`, the results are returned in an order corresponding to the order of the arguments.
+ `pool.apply(f, args)`: `f` is only executed in ONE of the workers of the pool. So ONE of the processes in the pool will run `f(args)`.
+ `pool.map(f, iterable)`: This method chops the iterable into a number of chunks which it submits to the process pool as separate tasks. So you take advantage of all the processes in the pool.

In [1]:
import multiprocessing as mp
import os, time, random

In [22]:
print("There is %s CPUs on this machine." %mp.cpu_count())

There is 4 CPUs on this machine.


In [3]:
def long_time_task(name):
    print('Run task %s (%s)...' % (name, os.getpid()))
    start = time.time()
    time.sleep(random.random() * 3)
    end = time.time()
    print('Task %s runs %0.2f seconds.' % (name, (end - start)))

if __name__=='__main__':
    print('Parent process %s.' % os.getpid())
    p = mp.Pool(4)
    for i in range(10):
        p.apply_async(long_time_task, args=(i,))
    print('Waiting for all subprocesses done...')
    p.close()
    p.join()
    print('All subprocesses done.')

Parent process 28490.
Run task 0 (28506)...
Run task 2 (28508)...
Run task 3 (28509)...
Run task 1 (28507)...
Waiting for all subprocesses done...
Task 1 runs 1.35 seconds.
Run task 4 (28507)...
Task 2 runs 1.46 seconds.
Run task 5 (28508)...
Task 3 runs 1.59 seconds.
Run task 6 (28509)...
Task 5 runs 0.23 seconds.
Run task 7 (28508)...
Task 4 runs 0.49 seconds.
Run task 8 (28507)...
Task 6 runs 0.95 seconds.
Run task 9 (28509)...
Task 0 runs 2.90 seconds.
Task 7 runs 1.31 seconds.
Task 8 runs 2.51 seconds.
Task 9 runs 2.24 seconds.
All subprocesses done.


In [12]:
# map()
def f(x):
    return x*x

p = mp.Pool(4)
p.map(f, [1, 2, 3])
# print(p.map(f, [1, 2, 3]))

[1, 4, 9]

## 3 - 多线程Threading
Reference: https://www.liaoxuefeng.com/wiki/0014316089557264a6b348958f449949df42a6d3a2e542c000/00143192823818768cd506abbc94eb5916192364506fa5d000
+ Python的标准库提供了两个模块：_thread和threading，_thread是低级模块，threading是高级模块，对_thread进行了封装。绝大多数情况下，我们只需要使用threading这个高级模块。
+ 启动一个线程就是把一个函数传入并创建Thread实例，然后调用start()开始执行
+ 由于任何进程默认就会启动一个线程，我们把该线程称为主线程，主线程又可以启动新的线程，Python的threading模块有个current_thread()函数，它永远返回当前线程的实例。主线程实例的名字叫MainThread，子线程的名字在创建时指定，我们用LoopThread命名子线程。

In [ ]:
import threading

# 新线程执行的代码:
def loop():
    print('thread %s is running...' % threading.current_thread().name)
    n = 0
    while n < 5:
        n = n + 1
        print('thread %s >>> %s' % (threading.current_thread().name, n))
        time.sleep(1)
    print('thread %s ended.' % threading.current_thread().name)

print('thread %s is running...' % threading.current_thread().name)
t = threading.Thread(target=loop, name='LoopThread')
t.start()
t.join()
print('thread %s ended.' % threading.current_thread().name) # Mainthread

## joblib

Reference: https://blog.dominodatalab.com/simple-parallelization/

In [26]:
from joblib import Parallel, delayed
import multiprocessing
     
# what are your inputs, and what operation do you want to 
# perform on each input. For example...
inputs = range(10) 
def processInput(i):
    print "yes"
    return i * i
 
num_cores = multiprocessing.cpu_count()

In [27]:
results = Parallel(n_jobs=num_cores)(delayed(processInput)(i) for i in inputs)
print results

[0, 1, 4, 9, 16, 25, 36, 49, 64, 81]
